In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
from openai import OpenAI

In [3]:
!conda list 

# packages in environment at C:\Users\vkbajoria\projects\agent-lang-graph\env:
#
# Name                    Version                   Build  Channel
anyio                     4.2.0           py312haa95532_0  
argon2-cffi               21.3.0             pyhd3eb1b0_0  
argon2-cffi-bindings      21.2.0          py312h2bbff1b_0  
asttokens                 2.0.5              pyhd3eb1b0_0  
async-lru                 2.0.4           py312haa95532_0  
attrs                     23.1.0          py312haa95532_0  
babel                     2.11.0          py312haa95532_0  
beautifulsoup4            4.12.3          py312haa95532_0  
blas                      1.0                         mkl  
bleach                    4.1.0              pyhd3eb1b0_0  
brotli                    1.0.9                h2bbff1b_8  
brotli-bin                1.0.9                h2bbff1b_8  
brotli-python             1.0.9           py312hd77b12b_8  
bzip2                     1.0.8                h2bbff1b_6  
ca-certifica

In [3]:
!conda list

# packages in environment at C:\Users\vkbajoria\projects\agent-lang-graph\env:
#
# Name                    Version                   Build  Channel
annotated-types           0.7.0                    pypi_0    pypi
anyio                     4.2.0           py312haa95532_0  
argon2-cffi               21.3.0             pyhd3eb1b0_0  
argon2-cffi-bindings      21.2.0          py312h2bbff1b_0  
asttokens                 2.0.5              pyhd3eb1b0_0  
async-lru                 2.0.4           py312haa95532_0  
attrs                     23.1.0          py312haa95532_0  
babel                     2.11.0          py312haa95532_0  
beautifulsoup4            4.12.3          py312haa95532_0  
blas                      1.0                         mkl  
bleach                    4.1.0              pyhd3eb1b0_0  
brotli                    1.0.9                h2bbff1b_8  
brotli-bin                1.0.9                h2bbff1b_8  
brotli-python             1.0.9           py312hd77b12b_8  
bzip2 

In [7]:
from openai import OpenAI

In [8]:
client = OpenAI()

In [9]:
chat_completion = client.chat.completions.create(model='gpt-3.5-turbo',messages=[{'role':'user','content':'Hello World'}])

In [10]:
chat_completion

ChatCompletion(id='chatcmpl-9fp4fjDruFbekk5cluYZ4KEJ5UeK9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1719754805, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))

In [54]:
class Agent:
    def __init__(self, system=''):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({'role': 'system', 'content':system})

    def __call__(self, message):
        self.messages.append({'role':'user', 'content':message})
        result = self.execute()
        self.messages.append({'role':'assistant', 'content': result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model='gpt-3.5-turbo',
            temperature=0,
            messages = self.messages
        )
        return completion.choices[0].message.content

In [55]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

dog_weight:
e.g. dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using dog_weight
Action: dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [56]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in 'Scottish Terrier':
        return (f'{name} average 20 lbs')
    elif name in 'Border Collie':
        return (f'{name} average weight is 34 lbs')
    elif name in 'Americal Bulldog':
        return (f'{name} average weight is 6 lbs')
    else:
        return (f'{name} average weight is 60 lbs')

known_actions = {
    'calculate': calculate,
    'dog_weight': average_dog_weight
}

In [71]:
average_dog_weight('Akita')

'Akita average weight is 60 lbs'

In [72]:
abot = Agent(prompt)

In [73]:
result = abot('How much does a Akita weight?')

In [74]:
result

'Thought: I should look up the weight of an Akita using the dog_weight action.\nAction: dog_weight: Akita\nPAUSE'

In [75]:
next_prompt = 'Observation: {}'.format(result)

In [76]:
abot(next_prompt)

'Answer: An Akita weighs around 100-130 lbs.'

In [53]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\ndog_weight:\ne.g. dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using dog_weight\nAction: dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a toy poodle weight?'},
 {'role': 

In [77]:
abot = Agent(prompt)

In [78]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I can find the weight of each dog using the dog_weight action and then calculate their combined weight.\n\nAction: dog_weight: Border Collie\nPAUSE'

In [82]:
import re
action_re = re.compile('^Action: (\w+): (.*)$')

<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:2: SyntaxWarning: invalid escape sequence '\w'
C:\Users\vkbajoria\AppData\Local\Temp\ipykernel_20348\2664302184.py:2: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)$')


In [86]:

def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        print(actions)
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception('Unknown action: {}: {}'.format(action, action_input))
            print(' -- running {} action with {} parameters'.format(action, action_input))
            observation = known_actions[action](action_input)
            print('Observation:', observation)
            next_prompt = 'Observation: {}'.format(observation)
        else:
            return

In [87]:
question = """I have 2 dogs, a border collie and a scottish terrier. What is their combined weight"""

In [88]:
query(question)

Thought: I need to find the weight of a Border Collie and a Scottish Terrier to calculate their combined weight.

Action: dog_weight: Border Collie
PAUSE
[<re.Match object; span=(0, 33), match='Action: dog_weight: Border Collie'>]
 -- running dog_weight action with Border Collie parameters
Observation: Border Collie average weight is 34 lbs
Thought: Now I need to find the weight of a Scottish Terrier.

Action: dog_weight: Scottish Terrier
PAUSE
[<re.Match object; span=(0, 36), match='Action: dog_weight: Scottish Terrier'>]
 -- running dog_weight action with Scottish Terrier parameters
Observation: Scottish Terrier average 20 lbs
Thought: Now that I have the weights of both dogs, I can calculate their combined weight.

Action: calculate: 34 + 20
PAUSE
[<re.Match object; span=(0, 26), match='Action: calculate: 34 + 20'>]
 -- running calculate action with 34 + 20 parameters
Observation: 54
Answer: The combined weight of a Border Collie and a Scottish Terrier is 54 lbs
[]
